In [ ]:
# 2025-11-18
from tqdm import tqdm
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
import requests
from dumper.utils import process_beacon
from normdata.utils import import_from_normdata
from AcdhArcheAssets.uri_norm_rules import get_normalized_uri 

In [ ]:
url = "https://raw.githubusercontent.com/acdh-oeaw/wiener-rundschau-data/refs/heads/main/wr-rundschau-authors.csv"
domain = "wiener-rundschau"
base_url = "https://acdh-oeaw.github.io/wiener-rundschau-static/"

In [ ]:
df = pd.read_csv(url)
df = df.dropna(subset=['gnd'])

In [ ]:
df.head()

In [ ]:
col, _ = Collection.objects.get_or_create(name="Wiener Rundschau")

In [ ]:
data = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    gnd = get_normalized_uri(row["gnd"])
    local_id = row["id"]
    beacon_uri = f"{base_url}{local_id}"
    try:
        item = Uri.objects.get(uri=gnd)
        entity = item.entity
    except ObjectDoesNotExist:
        entity = import_from_normdata(gnd, "person")
    item = {
        "id": local_id,
        "pmb_id": entity.id
    }
    data.append(item)
    try:
        entity.collection.add(col)
    except AttributeError:
        print(gnd, beacon_uri)
        continue
    try:
        Uri.objects.get(uri=beacon_uri)
        continue
    except ObjectDoesNotExist:
        new_uri = Uri.objects.create(uri=beacon_uri, entity=entity)
        new_uri.domain = domain
        new_uri.save()

In [ ]:
enriched_df = pd.DataFrame(data)

In [ ]:
merged_df = df.merge(enriched_df, on='id', how='left')

In [ ]:
merged_df.to_csv("merged-authors.csv", index=False)